In [29]:
# Open the text file for reading
with open("/Users/julianesler/Downloads/dv.txt", "r") as file:
    # Read the entire contents of the file
    file_contents = file.read()

# Now 'file_contents' contains the contents of the text file
print(file_contents)

agency_cd	site_no	datetime	100139_00060_00003	100139_00060_00003_cd
USGS	09368000	1934-10-01	750	A
USGS	09368000	1934-10-02	700	A
USGS	09368000	1934-10-03	650	A
USGS	09368000	1934-10-04	600	A
USGS	09368000	1934-10-05	550	A
USGS	09368000	1934-10-06	500	A
USGS	09368000	1934-10-07	450	A
USGS	09368000	1934-10-08	400	A
USGS	09368000	1934-10-09	400	A
USGS	09368000	1934-10-10	350	A
USGS	09368000	1934-10-11	330	A
USGS	09368000	1934-10-12	320	A
USGS	09368000	1934-10-13	310	A
USGS	09368000	1934-10-14	300	A
USGS	09368000	1934-10-15	310	A
USGS	09368000	1934-10-16	320	A
USGS	09368000	1934-10-17	342	A
USGS	09368000	1934-10-18	330	A
USGS	09368000	1934-10-19	390	A
USGS	09368000	1934-10-20	354	A
USGS	09368000	1934-10-21	310	A
USGS	09368000	1934-10-22	300	A
USGS	09368000	1934-10-23	280	A
USGS	09368000	1934-10-24	280	A
USGS	09368000	1934-10-25	240	A
USGS	09368000	1934-10-26	230	A
USGS	09368000	1934-10-27	206	A
USGS	09368000	1934-10-28	190	A
USGS	09368000	1934-10-29	174	A
USGS	09368000	1934-10-30	198	A
US

In [30]:
import csv

# Specify the input and output file names
input_file = '/Users/julianesler/Downloads/dv.txt'
output_file = 'output.csv'

# Specify the delimiter used in the input file
delimiter = '\t'  # Change this to the actual delimiter used in your file

# Open the input file in text mode and the output file in CSV mode
with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
    # Create a CSV writer object
    csv_writer = csv.writer(outfile)

    # Read each line from the input file and write it to the CSV file
    for line in infile:
        # Use the correct delimiter to split the line into fields
        fields = line.strip().split(delimiter)
        
        # Write the fields to the CSV file
        csv_writer.writerow(fields)

# Open and print the contents of the output file
with open(output_file, 'r') as printed_file:
    content = printed_file.read()
    print(content)

agency_cd,site_no,datetime,100139_00060_00003,100139_00060_00003_cd
USGS,09368000,1934-10-01,750,A
USGS,09368000,1934-10-02,700,A
USGS,09368000,1934-10-03,650,A
USGS,09368000,1934-10-04,600,A
USGS,09368000,1934-10-05,550,A
USGS,09368000,1934-10-06,500,A
USGS,09368000,1934-10-07,450,A
USGS,09368000,1934-10-08,400,A
USGS,09368000,1934-10-09,400,A
USGS,09368000,1934-10-10,350,A
USGS,09368000,1934-10-11,330,A
USGS,09368000,1934-10-12,320,A
USGS,09368000,1934-10-13,310,A
USGS,09368000,1934-10-14,300,A
USGS,09368000,1934-10-15,310,A
USGS,09368000,1934-10-16,320,A
USGS,09368000,1934-10-17,342,A
USGS,09368000,1934-10-18,330,A
USGS,09368000,1934-10-19,390,A
USGS,09368000,1934-10-20,354,A
USGS,09368000,1934-10-21,310,A
USGS,09368000,1934-10-22,300,A
USGS,09368000,1934-10-23,280,A
USGS,09368000,1934-10-24,280,A
USGS,09368000,1934-10-25,240,A
USGS,09368000,1934-10-26,230,A
USGS,09368000,1934-10-27,206,A
USGS,09368000,1934-10-28,190,A
USGS,09368000,1934-10-29,174,A
USGS,09368000,1934-10-30,198,A
US

In [31]:
import pandas as pd

# Specify the path to your CSV file
csv_file_path = 'output.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

# Now 'df' is a pandas DataFrame containing your CSV data
print(df)

      agency_cd  site_no    datetime  100139_00060_00003 100139_00060_00003_cd
0          USGS  9368000  1934-10-01               750.0                     A
1          USGS  9368000  1934-10-02               700.0                     A
2          USGS  9368000  1934-10-03               650.0                     A
3          USGS  9368000  1934-10-04               600.0                     A
4          USGS  9368000  1934-10-05               550.0                     A
...         ...      ...         ...                 ...                   ...
32551      USGS  9368000  2023-11-14               639.0                     P
32552      USGS  9368000  2023-11-15               636.0                     P
32553      USGS  9368000  2023-11-16               639.0                     P
32554      USGS  9368000  2023-11-17               629.0                     P
32555      USGS  9368000  2023-11-18               617.0                     P

[32556 rows x 5 columns]


In [32]:
df.rename(columns={'agency_cd': 'Agency'}, inplace=True)
df.rename(columns={'site_no': 'SiteID'}, inplace=True)
df.rename(columns={'datetime': 'Date'}, inplace=True)
df.rename(columns={'100139_00060_00003': 'Streamflow'}, inplace=True)
df.rename(columns={'100139_00060_00003_cd': 'Grade'}, inplace=True)
print(df)

      Agency   SiteID        Date  Streamflow Grade
0       USGS  9368000  1934-10-01       750.0     A
1       USGS  9368000  1934-10-02       700.0     A
2       USGS  9368000  1934-10-03       650.0     A
3       USGS  9368000  1934-10-04       600.0     A
4       USGS  9368000  1934-10-05       550.0     A
...      ...      ...         ...         ...   ...
32551   USGS  9368000  2023-11-14       639.0     P
32552   USGS  9368000  2023-11-15       636.0     P
32553   USGS  9368000  2023-11-16       639.0     P
32554   USGS  9368000  2023-11-17       629.0     P
32555   USGS  9368000  2023-11-18       617.0     P

[32556 rows x 5 columns]


In [44]:
df['Year'] = df['Date'].dt.year
print(df)

      Agency   SiteID       Date  Streamflow Grade  Year
0       USGS  9368000 1934-10-01       750.0     A  1934
1       USGS  9368000 1934-10-02       700.0     A  1934
2       USGS  9368000 1934-10-03       650.0     A  1934
3       USGS  9368000 1934-10-04       600.0     A  1934
4       USGS  9368000 1934-10-05       550.0     A  1934
...      ...      ...        ...         ...   ...   ...
32551   USGS  9368000 2023-11-14       639.0     P  2023
32552   USGS  9368000 2023-11-15       636.0     P  2023
32553   USGS  9368000 2023-11-16       639.0     P  2023
32554   USGS  9368000 2023-11-17       629.0     P  2023
32555   USGS  9368000 2023-11-18       617.0     P  2023

[32556 rows x 6 columns]


In [54]:
# Convert 'Date' to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Sort DataFrame by date (if not sorted already)
df.sort_values(by='Date', inplace=True)

# Add a new column for the rolling 7-day average
df['7DayMean'] = df['Streamflow'].rolling(window=7).mean()

# Display the result
print(df)

      Agency   SiteID       Date  Streamflow Grade  Year    7DayMean
0       USGS  9368000 1934-10-01       750.0     A  1934         NaN
1       USGS  9368000 1934-10-02       700.0     A  1934         NaN
2       USGS  9368000 1934-10-03       650.0     A  1934         NaN
3       USGS  9368000 1934-10-04       600.0     A  1934         NaN
4       USGS  9368000 1934-10-05       550.0     A  1934         NaN
...      ...      ...        ...         ...   ...   ...         ...
32551   USGS  9368000 2023-11-14       639.0     P  2023  646.285714
32552   USGS  9368000 2023-11-15       636.0     P  2023  647.285714
32553   USGS  9368000 2023-11-16       639.0     P  2023  645.571429
32554   USGS  9368000 2023-11-17       629.0     P  2023  641.285714
32555   USGS  9368000 2023-11-18       617.0     P  2023  636.000000

[32556 rows x 7 columns]


In [57]:
print(df.loc[df.index[7]])

Agency                       USGS
SiteID                    9368000
Date          1934-10-08 00:00:00
Streamflow                  400.0
Grade                           A
Year                         1934
7DayMean                    550.0
Name: 7, dtype: object


In [101]:
import math
import numpy as np

#gdf stands for grouped daily flow or perhaps grouped data frame.
gdf = df.groupby('Year')['7DayMean'].agg(list).reset_index()

# Rename the columns if needed
gdf.columns = ['Year', '7DayMean']

ls = gdf.loc[0,'7DayMean']
ls1 = [Q for Q in ls if not math.isnan(Q)]
print (ls1)

gdf.at[0,'7DayMean'] = ls1
# Display the result
print(gdf)

[600.0, 550.0, 507.14285714285717, 464.2857142857143, 425.7142857142857, 392.85714285714283, 365.7142857142857, 344.2857142857143, 331.42857142857144, 320.0, 318.85714285714283, 318.85714285714283, 328.85714285714283, 335.14285714285717, 336.57142857142856, 335.14285714285717, 329.42857142857144, 320.57142857142856, 307.7142857142857, 284.85714285714283, 263.7142857142857, 246.57142857142858, 228.57142857142858, 216.85714285714286, 201.71428571428572, 195.71428571428572, 198.57142857142858, 202.0, 212.0, 232.85714285714286, 255.14285714285714, 284.2857142857143, 311.7142857142857, 333.42857142857144, 351.14285714285717, 358.2857142857143, 363.14285714285717, 373.42857142857144, 375.14285714285717, 371.7142857142857, 370.0, 375.14285714285717, 395.14285714285717, 410.85714285714283, 410.85714285714283, 425.42857142857144, 435.7142857142857, 446.2857142857143, 446.2857142857143, 437.7142857142857, 428.85714285714283, 416.85714285714283, 395.42857142857144, 377.14285714285717, 350.8571428

In [104]:
# Find the minimum value for each list in the '7DayMean' column
gdf['AnnualMin'] = gdf['7DayMean'].apply(lambda x: min(x) if x else None)

# Display the result
print(gdf)

    Year                                           7DayMean   AnnualMin
0   1934  [600.0, 550.0, 507.14285714285717, 464.2857142...  195.714286
1   1935  [479.14285714285717, 474.0, 474.0, 493.4285714...  380.285714
2   1936  [624.0, 624.0, 605.1428571428571, 584.85714285...  312.000000
3   1937  [683.5714285714286, 628.1428571428571, 584.142...  140.285714
4   1938  [548.2857142857143, 552.1428571428571, 554.714...  254.714286
..   ...                                                ...         ...
85  2019  [501.2857142857143, 513.2857142857143, 521.571...  479.571429
86  2020  [867.1428571428571, 847.2857142857143, 832.428...  282.428571
87  2021  [492.14285714285717, 487.85714285714283, 480.8...  333.857143
88  2022  [554.7142857142857, 533.7142857142857, 523.571...  349.142857
89  2023  [587.8571428571429, 612.1428571428571, 624.285...  412.285714

[90 rows x 3 columns]


In [109]:
gdf['SortedAnnualMin'] = gdf['AnnualMin'].sort_values().tolist()
print(gdf)

low_flow = gdf['SortedAnnualMin'].quantile(0.1)

# Display the result
print("7Q10 Low Flow (CF/S):", low_flow)


    Year                                           7DayMean    AnnualMin  \
0   1934  [600.0, 550.0, 507.14285714285717, 464.2857142...    12.714286   
1   1935  [479.14285714285717, 474.0, 474.0, 493.4285714...    12.857143   
2   1936  [624.0, 624.0, 605.1428571428571, 584.85714285...    16.714286   
3   1937  [683.5714285714286, 628.1428571428571, 584.142...    52.285714   
4   1938  [548.2857142857143, 552.1428571428571, 554.714...    65.857143   
..   ...                                                ...          ...   
85  2019  [501.2857142857143, 513.2857142857143, 521.571...   873.714286   
86  2020  [867.1428571428571, 847.2857142857143, 832.428...   933.571429   
87  2021  [492.14285714285717, 487.85714285714283, 480.8...   973.857143   
88  2022  [554.7142857142857, 533.7142857142857, 523.571...  1415.714286   
89  2023  [587.8571428571429, 612.1428571428571, 624.285...  1594.285714   

    SortedAnnualMin  
0         12.714286  
1         12.857143  
2         16.714286  